Results of this one were just demonstrative, virtually useless. The next one, `B5_2`, makes a comprehensivre tree.

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt


def log2(p):
    return np.ma.log2(p).filled(0)


def dataframe_entropy(d: pd.DataFrame, column: str):
    return entropy_singular(d[column].value_counts()[0] / len(d))


def entropy(p):
    ap = 1-p
    return -(p*log2(p) + ap*log2(ap))


def entropy_singular(p):
    if p == 0 or p == 1:
        return 0
    ap = 1-p
    return -(p*np.log2(p) + ap*np.log2(ap))


raw_dataset = pd.read_csv("./dogscats.csv")
raw_dataset.tail()


,Ears shape,Face shape,Whiskers,Cat
5,Pointy,Round,Absent,Yes
6,Floppy,Not round,Absent,No
7,Pointy,Round,Absent,Yes
8,Floppy,Round,Absent,No
9,Floppy,Round,Absent,No


In [2]:
dataset = raw_dataset.copy()


def dt_split(ds: pd.DataFrame, by: str, ignore: list[str] = [], verbose=False):
    uniques = ds[by].unique()
    ruler = uniques[0]
    BASE_ENTROPY = dataframe_entropy(ds, by)
    if verbose:
        print("")
        print("")
        print(f"----==========Splitting:==========----")
        print(f"Unique values in {by}: {uniques}")
        print(f"BASE ENTROPY: {BASE_ENTROPY}")
        if ignore:
            print(f"Parent: {ignore[-1]} (if '{ds[ignore[-1]].unique()[0]}')")
        else:
            print(f"Parent: <ROOT>")
    SPLIT_ENTROPIES_VERBOSE = {}
    SPLIT_ENTROPIES = []
    for col in ds.columns[:-1]:
        if col in ignore:
            continue
        (dgr1n, dgr1), (dgr2n, dgr2) = ds.groupby(col)
        entr1 = dataframe_entropy(dgr1, by)
        entr2 = dataframe_entropy(dgr2, by)
        avg = (entr1 * len(dgr1) + entr2 * len(dgr2)) / len(ds)
        reduction = BASE_ENTROPY - avg
        SPLIT_ENTROPIES.append([col, reduction])
        SPLIT_ENTROPIES_VERBOSE[col] = (entr1, entr2, avg, reduction)
        if verbose:
            print("--------------------------")
            print(f"{col}:")
            vc = dgr1[by].value_counts()
            print(f"{vc[ruler] if ruler in vc else 0}/{len(dgr1)} {dgr1n} --> E {entr1}")
            vc = dgr2[by].value_counts()
            print(f"{vc[ruler] if ruler in vc else 0}/{len(dgr2)} {dgr2n} --> E {entr2}")
            print(f"A {avg}")
            print(f"R {reduction}")
            print("--------------------------")
    SPLIT_ENTROPIES = np.array(SPLIT_ENTROPIES)
    winner = SPLIT_ENTROPIES[np.argmax(SPLIT_ENTROPIES[:,1])][0]
    sufficient = SPLIT_ENTROPIES_VERBOSE[winner][2] == 0
    return winner, sufficient, SPLIT_ENTROPIES_VERBOSE

def recursive_split(ds: pd.DataFrame, by: str, attach_to: dict|None=None, ignore: list[str]=[], verbose=False):
    if attach_to is None:
        attach_to = {}
    w, ws, wv = dt_split(ds=ds,by=by,ignore=ignore,verbose=verbose)
    print(f"Winner: {w} ({'Sufficient' if ws else 'Not sufficient'})")
    if ws:
        attach_to[w] = None
    else:
        attach_to[w] = {}
        s1,s2 = ds.groupby(w)
        print(f"Splitting by {w}: 1.{s1[0]} and 2.{s2[0]}")
        recursive_split(ds=s1[1],by=by,attach_to=attach_to[w],ignore=[*ignore,w],verbose=verbose)
        recursive_split(ds=s2[1],by=by,attach_to=attach_to[w],ignore=[*ignore,w],verbose=verbose)
    return attach_to

tree = recursive_split(dataset,"Cat", verbose=True)
print("OPTIMAL CRITERIA TREE (UNORDERED):")
print(tree)



----==========Splitting:==========----
Unique values in Cat: ['Yes' 'No']
BASE ENTROPY: 1.0
Parent: <ROOT>
--------------------------
Ears shape:
1/5 Floppy --> E 0.7219280948873623
4/5 Pointy --> E 0.7219280948873623
A 0.7219280948873623
R 0.2780719051126377
--------------------------
--------------------------
Face shape:
1/3 Not round --> E 0.9182958340544896
4/7 Round --> E 0.9852281360342515
A 0.965148445440323
R 0.034851554559677034
--------------------------
--------------------------
Whiskers:
2/6 Absent --> E 0.9182958340544896
3/4 Present --> E 0.8112781244591328
A 0.8754887502163469
R 0.12451124978365313
--------------------------
Winner: Ears shape (Not sufficient)
Splitting by Ears shape: 1.Floppy and 2.Pointy


----==========Splitting:==========----
Unique values in Cat: ['Yes' 'No']
BASE ENTROPY: 0.7219280948873623
Parent: Ears shape (if 'Floppy')
--------------------------
Face shape:
1/2 Not round --> E 1.0
0/3 Round --> E 0
A 0.4
R 0.3219280948873623
---------------